In [7]:
import sqlalchemy as sa
import pandas as pd

import json
import logging
import requests

from sqlalchemy.sql import text
from sqlalchemy import inspect

In [10]:
def get_database_engine(environment: str):
    if environment == 'production':
        url = 'https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json'
        params = requests.get(url).json()['params']
        engine = sa.create_engine(
            f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}"
        )
    
    elif environment == 'staging':
        with open('alerceread_db_staging.json', 'r', encoding='utf-8') as f:
            params = json.load(f)
        engine = sa.create_engine(
            f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}:{params['port']}/{params['dbname']}"
        )
    else:
        raise ValueError(f'Environment "{environment}" not defined')
        
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    print('Available Tables:\n', tables)
    return engine

In [11]:
ENVIRONMENT = "production"
engine = get_database_engine(environment=ENVIRONMENT)
engine.begin()

Available Tables:
 ['outlier', 'feature_version', 'gaia_ztf', 'alembic_version', 'ss_ztf', 'allwise', 'non_detection', 'step', 'taxonomy', 'feature', 'pipeline', 'lc_classifier_top', 'lc_classifier_periodic', 'lc_classifier_transient', 'magstat', 'lc_classifier', 'stamp_classifier', 'dataquality', 'object', 'lc_classifier_stochastic', 'lc_classifier_beta', 'forced_photometry', 'lc_classifier_bhrf_forced_phot_childs', 'reference', 'ps1_ztf', 'xmatch', 'detection', 'probability']


In [4]:
from astropy.time import Time

mjd = Time("2025-03-17").mjd
mjd

np.float64(60751.0)

In [5]:
# Construimos la consulta SQL
query = f"""
    SELECT * FROM object 
    WHERE lastmjd >= {mjd};
"""

# Ejecutamos la consulta
objects = pd.read_sql_query(query, con=engine)

display(objects.head())
print(objects.shape)
print(objects.columns)

,oid,ndethist,ncovhist,mjdstarthist,mjdendhist,corrected,stellar,ndet,g_r_max,g_r_max_corr,...,meanra,meandec,sigmara,sigmadec,deltajd,firstmjd,lastmjd,step_id_corr,diffpos,reference_change
0,ZTF21aaaudqn,10,869,59218.326053,60751.222164,True,True,2,None,None,...,129.928115,84.968703,0.588749,0.051633,12.007569,60739.214595,60751.222164,27.4.1a4,True,False
1,ZTF22abvvgme,433,3035,58903.265706,60751.178229,True,True,12,None,None,...,126.162298,45.153020,0.029258,0.020633,44.899398,60706.278831,60751.178229,27.4.1a4,True,False
2,ZTF18acgtxps,1277,2101,58095.364931,60751.259201,True,True,12,None,None,...,110.263945,53.314774,0.036080,0.021555,45.037187,60706.222014,60751.259201,27.4.1a4,True,False
3,ZTF22abjxsiz,272,2113,59135.436424,60751.259201,True,True,12,None,None,...,114.539743,52.295399,0.035244,0.021555,45.037187,60706.222014,60751.259201,27.4.1a4,False,False
4,ZTF19acfahzt,4,2130,58773.094653,60751.479375,True,True,1,None,None,...,280.874210,8.922112,0.065796,0.065000,0.000000,60751.479375,60751.479375,27.4.1a4,False,False


(247893, 22)
Index(['oid', 'ndethist', 'ncovhist', 'mjdstarthist', 'mjdendhist',
       'corrected', 'stellar', 'ndet', 'g_r_max', 'g_r_max_corr', 'g_r_mean',
       'g_r_mean_corr', 'meanra', 'meandec', 'sigmara', 'sigmadec', 'deltajd',
       'firstmjd', 'lastmjd', 'step_id_corr', 'diffpos', 'reference_change'],
      dtype='object')


## OIDs que no son bogus y tienen igual o más de 8 detecciones

In [31]:
oids = list(objects.oid.values[:2])
oids = ['ZTF24aafnfsk']
print(f'Num. oids: {len(oids)}')

oids = [f"'{oid}'" for oid in oids]
query_detections = f"""
    SELECT * FROM detection
    WHERE oid IN ({','.join(oids)}) and rb >= 0.55;
"""

detections = pd.read_sql_query(query_detections, con=engine)
oids_mayor_8 = detections['oid'].value_counts()
oids = oids_mayor_8[oids_mayor_8 >= 8].index
print(f'Num. oids ndet >= 8: {len(oids)}')

oids = [f"'{oid}'" for oid in oids]
print(oids)

Num. oids: 1


Num. oids ndet >= 8: 1
["'ZTF24aafnfsk'"]


## Query to BBDD

### Detections and Forced Photometry

In [32]:
query_detections = f"""
    SELECT * FROM detection
    WHERE oid IN ({','.join(oids)}) and rb >= 0.55;
"""

detections = pd.read_sql_query(query_detections, con=engine)
display(detections.head())
print(detections.shape)
print(detections.columns)

,oid,candid,mjd,fid,pid,diffmaglim,isdiffpos,nid,ra,dec,...,sigmagapbig,rfid,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext,corrected,dubious,parent_candid,has_stamp,step_id_corr
0,ZTF24aafnfsk,2624212900415015008,60378.212905,2,2624212900415,20.406800,1,2624,118.586843,16.598465,...,0.3873,NaN,18.768087,0.042888,0.050690,True,False,2.650154e+18,False,23.12.x
1,ZTF24aafnfsk,2637193980415015005,60391.193981,1,2637193980415,19.558700,1,2637,118.586974,16.598398,...,0.4302,NaN,19.269058,0.135073,0.135867,True,False,2.650154e+18,False,23.12.x
2,ZTF24aafnfsk,2645154030415015005,60399.154039,1,2645154030415,20.076600,1,2645,118.587022,16.598453,...,0.4032,NaN,19.312847,0.111133,0.112177,True,False,2.650154e+18,False,23.12.x
3,ZTF24aafnfsk,2650154170415015010,60404.154178,2,2650154170415,20.465046,1,2650,118.587029,16.598448,...,0.3115,566120204.0,18.732126,0.038569,0.046593,True,False,NaN,True,23.12.x
4,ZTF24aafnfsk,2650172320415015007,60404.172326,1,2650172320415,20.383709,1,2650,118.587070,16.598486,...,0.2644,566120104.0,19.220999,0.092241,0.093302,True,False,NaN,True,23.12.x


(21, 30)
Index(['oid', 'candid', 'mjd', 'fid', 'pid', 'diffmaglim', 'isdiffpos', 'nid',
       'ra', 'dec', 'magpsf', 'sigmapsf', 'magap', 'sigmagap', 'distnr', 'rb',
       'rbversion', 'drb', 'drbversion', 'magapbig', 'sigmagapbig', 'rfid',
       'magpsf_corr', 'sigmapsf_corr', 'sigmapsf_corr_ext', 'corrected',
       'dubious', 'parent_candid', 'has_stamp', 'step_id_corr'],
      dtype='object')


In [33]:
# FORCED PHOTOMETRY
query_forced_photometry = f"""
                    SELECT * FROM forced_photometry
                    WHERE oid in ({','.join(oids)}) and procstatus in ('0', '57');
                    """

forced_photometry = pd.read_sql_query(query_forced_photometry, con=engine)
display(forced_photometry.head())
print(forced_photometry.shape)
print(forced_photometry.columns)

,pid,oid,mjd,fid,ra,dec,e_ra,e_dec,mag,e_mag,...,diffmaglim,programid,procstatus,distnr,ranr,decnr,magnr,sigmagnr,chinr,sharpnr
0,2970304600415,ZTF24aafnfsk,60724.304607,2,118.5871,16.598508,None,None,20.280020,0.180514,...,19.987000,1,0,0.287778,118.587036,16.598562,19.049,0.035,2.209,0.078
1,2972360420415,ZTF24aafnfsk,60726.360428,1,118.5871,16.598508,None,None,20.345940,0.135271,...,20.325899,1,0,0.212998,118.587044,16.598536,20.046,0.030,0.982,0.040
2,2972431720415,ZTF24aafnfsk,60726.431724,2,118.5871,16.598508,None,None,20.431459,0.279470,...,19.599800,1,0,0.287778,118.587036,16.598562,19.049,0.035,2.209,0.078
3,2974255270415,ZTF24aafnfsk,60728.255278,1,118.5871,16.598508,None,None,20.357624,0.103662,...,20.664499,1,0,0.212998,118.587044,16.598536,20.046,0.030,0.982,0.040
4,2976170890415,ZTF24aafnfsk,60730.170891,2,118.5871,16.598508,None,None,20.349798,0.131217,...,20.268000,1,0,0.287778,118.587036,16.598562,19.049,0.035,2.209,0.078


(47, 42)
Index(['pid', 'oid', 'mjd', 'fid', 'ra', 'dec', 'e_ra', 'e_dec', 'mag',
       'e_mag', 'mag_corr', 'e_mag_corr', 'e_mag_corr_ext', 'isdiffpos',
       'corrected', 'dubious', 'parent_candid', 'has_stamp', 'field', 'rcid',
       'rfid', 'sciinpseeing', 'scibckgnd', 'scisigpix', 'magzpsci',
       'magzpsciunc', 'magzpscirms', 'clrcoeff', 'clrcounc', 'exptime',
       'adpctdif1', 'adpctdif2', 'diffmaglim', 'programid', 'procstatus',
       'distnr', 'ranr', 'decnr', 'magnr', 'sigmagnr', 'chinr', 'sharpnr'],
      dtype='object')


### To compute Features locally

In [34]:
def patch_wise(oid_list):
    columns = ["oid", "w1mpro", "w2mpro", "w3mpro", "w4mpro"]
    df = pd.DataFrame({"oid": oid_list})  # Fill only the oid column
    for col in columns[1:]:  # Add empty columns for the rest
        df[col] = None
    return df

In [40]:
# REFERENCE
query_reference = f"""
                    SELECT oid, rfid, sharpnr, chinr FROM reference
                    WHERE oid in ({','.join(oids)}) and chinr >= 0;
                    """
reference = pd.read_sql_query(query_reference, con=engine)
#reference = reference.drop_duplicates("rfid")

# XMATCH
query_xmatch = f"""
                SELECT oid, oid_catalog, dist FROM xmatch
                WHERE oid in ({','.join(oids)}) and catid='allwise';
                """
xmatch = pd.read_sql_query(query_xmatch, con=engine)
xmatch = xmatch.sort_values("dist").drop_duplicates("oid")
oid_catalog = [f"'{oid}'" for oid in xmatch["oid_catalog"].values]

#########################################################################################
# ALL WISE
query_wise = f"""
            SELECT oid_catalog, w1mpro, w2mpro, w3mpro, w4mpro FROM allwise
            WHERE oid_catalog in ({','.join(oid_catalog)});
            """
try:
    wise = pd.read_sql_query(query_wise, con=engine).set_index("oid_catalog")
    wise = pd.merge(xmatch, wise, on="oid_catalog", how="outer")
    wise = wise[["oid", "w1mpro", "w2mpro", "w3mpro", "w4mpro"]].set_index("oid")
except:
    wise = patch_wise(oids)
    logging.warning("wise patched")

# PANSTARR
query_ps = f"""
            SELECT oid, sgscore1, sgmag1, srmag1, simag1, szmag1, distpsnr1 FROM ps1_ztf
            WHERE oid in ({','.join(oids)});
            """
ps = pd.read_sql_query(query_ps, con=engine)
ps = ps.drop_duplicates("oid").set_index("oid")
#########################################################################################

xmatch = pd.concat([wise, ps], axis=1).reset_index()

In [43]:
import numpy as np
from lc_classifier.utils import (
    all_features_from_astro_objects,
    create_astro_object,
    EmptyLightcurveException,
)

def patch_xmatch_by_oid(oid: str):
    data = {
        "oid": [oid],
        "w1mpro": [np.nan],
        "w2mpro": [np.nan],
        "w3mpro": [np.nan],
        "w4mpro": [np.nan],
        "sgscore1": [np.nan],
        "sgmag1": [np.nan],
        "srmag1": [np.nan],
        "simag1": [np.nan],
        "szmag1": [np.nan],
        "distpsnr1": [np.nan],
    }
    return pd.DataFrame(data)

def dataframes_to_astro_object_list(
    detections,
    forced_photometry,
    xmatch,
    reference,
    features=None,
    data_origin="database",
    verbose=True,
):
    oids = detections["oid"].unique()
    detections.set_index("oid", inplace=True)
    detections.sort_index(inplace=True)
    forced_photometry_columns = forced_photometry.columns
    forced_photometry.set_index("oid", inplace=True)
    forced_photometry.sort_index(inplace=True)
    xmatch.set_index("oid", inplace=True)
    reference_columns = reference.columns
    reference.set_index("oid", inplace=True)
    astro_objects_list = []
    for oid in oids:
        try:
            xmatch_oid = xmatch.loc[[oid]].reset_index()
        except:
            logging.warning("xmatch metadata patched!")
            xmatch_oid = patch_xmatch_by_oid(oid=oid).reset_index()

        assert len(xmatch_oid) == 1
        xmatch_oid = xmatch_oid.iloc[0]
        if oid in forced_photometry.index:
            forced_photometry_oid = forced_photometry.loc[[oid]].reset_index()
        else:
            forced_photometry_oid = pd.DataFrame(columns=forced_photometry_columns)
        if oid in reference.index:
            reference_oid = reference.loc[[oid]].reset_index()
        else:
            reference_oid = pd.DataFrame(columns=reference_columns)
        try:
            ao = create_astro_object(
                data_origin=data_origin,
                detections=detections.loc[[oid]].reset_index(),
                forced_photometry=forced_photometry_oid,
                xmatch=xmatch_oid,
                reference=reference_oid,
                non_detections=None,
            )
            if features is not None:
                """add features from db"""
                try:
                    ao.features = features.loc[features.oid == oid][
                        ["name", "value", "fid", "version"]
                    ]
                except:
                    ao.features = None
        except EmptyLightcurveException:
            continue

        astro_objects_list.append(ao)
    return astro_objects_list

ImportError: Numba needs NumPy 1.26 or less

In [ ]:
a = a[(a["mag"] != 100) | (a["e_mag"] != 100)].copy()
a.rename(
    columns={"mag_corr": "brightness", "e_mag_corr_ext": "e_brightness"},
    inplace=True,
)
a["unit"] = "magnitude"

In [41]:
reference

,oid,rfid,sharpnr,chinr
0,ZTF24aafnfsk,566120104,0.04,0.982


In [37]:
xmatch

,oid,w1mpro,w2mpro,w3mpro,w4mpro,sgscore1,sgmag1,srmag1,simag1,szmag1,distpsnr1
0,ZTF24aafnfsk,NaN,NaN,NaN,NaN,0.000625,19.5982,19.0322,18.8284,18.5152,0.103234


In [206]:
# GET FEATURES
query_features = f"""
                SELECT * FROM feature as f 
                WHERE f.oid in ({','.join(oids)}) and f.version = '27.4.1a4';
                """

features = pd.read_sql_query(query_features, con=engine)
features

,oid,name,value,fid,version
0,ZTF18acbvugo,Amplitude,NaN,1,27.4.1a4
1,ZTF18acbvugo,Amplitude,NaN,2,27.4.1a4
2,ZTF18acbvugo,AndersonDarling,NaN,1,27.4.1a4
3,ZTF18acbvugo,AndersonDarling,NaN,2,27.4.1a4
4,ZTF18acbvugo,Autocor_length,NaN,1,27.4.1a4
...,...,...,...,...,...
204,ZTF18acbvugo,ulens_t0,2.396655e+01,2,27.4.1a4
205,ZTF18acbvugo,ulens_tE,1.121472e+01,1,27.4.1a4
206,ZTF18acbvugo,ulens_tE,5.473820e-01,2,27.4.1a4
207,ZTF18acbvugo,ulens_u0,1.526989e+02,1,27.4.1a4


In [207]:
def features_from_db(features: pd.DataFrame):
    def normalize_name(x: str | int):
        if "_0" in x:
            return x[:-2].replace("/", "_")
        return x.replace("/", "_")

    def normalize_fid(x):
        if x == 0:
            return None
        if x == 1:
            return "g"
        if x == 2:
            return "r"
        if x == 12:
            return "g,r"

    # features['name'] = features['name'] + '_' + features['fid'].astype(str)
    if len(features) >= 1:
        features["name"] = features.apply(lambda x: normalize_name(x["name"]), axis=1)
        features["fid"] = features.apply(lambda x: normalize_fid(x["fid"]), axis=1)
        return features
    else:
        return None

In [208]:
#features = features_from_db(features)
#features

In [209]:
data_dict = {
    "detections": detections,
    "forced_photometry": forced_photometry,
    "features": features,
    "xmatch": None,
    "reference": None,
}

In [210]:
import numpy as np

mag_corr_column = "magpsf_corr"
e_mag_corr_ext_column = "sigmapsf_corr_ext"
diff_mag_column = "magpsf"
e_diff_mag_column = "sigmapsf"
mag_corr_column_fp = "mag_corr"
e_mag_corr_ext_column_fp = "e_mag_corr_ext"
diff_mag_column_fp = "mag"
e_diff_mag_column_fp = "e_mag"
w1_column = "w1mpro"
w2_column = "w2mpro"
w3_column = "w3mpro"
w4_column = "w4mpro"


detection_keys = [
    "oid",
    "candid",
    "pid",
    "ra",
    "dec",
    "mjd",
    mag_corr_column,
    e_mag_corr_ext_column,
    diff_mag_column,
    e_diff_mag_column,
    "fid",
    "isdiffpos",
]

forced_photometry_keys = [
    "oid",
    "candid",
    "pid",
    "ra",
    "dec",
    "mjd",
    mag_corr_column_fp,
    e_mag_corr_ext_column_fp,
    diff_mag_column_fp,
    e_diff_mag_column_fp,
    "fid",
    "isdiffpos",
]

def mag2flux(mag):
    return 10 ** (-(mag - 23.9) / 2.5)

def mag_err_2_flux_err(mag_err, mag):
    return np.log(10.0) * mag2flux(mag) / 2.5 * mag_err

def recover_mag_and_emag_with_signed_flux(df):
    df["isdiffpos"] = np.sign(df["fluxdiff_uJy"]).astype(int)
    df["mag"] = -2.5 * np.log10(np.abs(df["fluxdiff_uJy"])) + 23.9
    df["e_mag"] = df["fluxerrdiff_uJy"] / np.abs(df["fluxdiff_uJy"])
    df.loc[df["fluxdiff_uJy"] == 0, ["mag", "e_mag"]] = np.nan
    return df

def get_det_AO_to_InputDTO(lightcurve):
    detections = lightcurve.copy()
    detections = detections[detections.unit == "diff_flux"].rename(
        columns={
            "brightness": "fluxdiff_uJy",
            "e_brightness": "fluxerrdiff_uJy",
        }
    )
    detections = recover_mag_and_emag_with_signed_flux(detections)
    return detections

def preprocess_detections_from_db(detections: pd.DataFrame) -> pd.DataFrame:

    detections_ = detections.copy()
    detections_.fillna(value=np.nan, inplace=True)
    detections_["forced"] = False
    detections_["unit"] = "diff_flux"

    detections_.rename(
        columns={
            mag_corr_column: "mag_corr",
            e_mag_corr_ext_column: "e_mag_corr_ext",
            diff_mag_column: "mag",
            e_diff_mag_column: "e_mag",
        },
        inplace=True,
    )

    detections_.rename(
        columns={"mag_corr": "brightness", "e_mag_corr_ext": "e_brightness"},
        inplace=True,
    )

    detections_["brightness"] = mag2flux(detections_["mag"]) * detections_["isdiffpos"]
    detections_["e_brightness"] = mag_err_2_flux_err(
        detections_["e_mag"], detections_["mag"]
    )
    return detections_

def preprocess_forced_photometry_from_db(
    forced_photometry: pd.DataFrame,
) -> pd.DataFrame:

    forced_photometry_ = forced_photometry.copy()
    forced_photometry_.fillna(value=np.nan, inplace=True)
    forced_photometry_["unit"] = "diff_flux"
    forced_photometry_.rename(
        columns={
            mag_corr_column_fp: "mag_corr",
            e_mag_corr_ext_column_fp: "e_mag_corr_ext",
            diff_mag_column_fp: "mag",
            e_diff_mag_column_fp: "e_mag",
        },
        inplace=True,
    )

    forced_photometry_.rename(
        columns={"mag_corr": "brightness", "e_mag_corr_ext": "e_brightness"},
        inplace=True,
    )

    forced_photometry_["brightness"] = (
        mag2flux(forced_photometry_["mag"]) * forced_photometry_["isdiffpos"]
    )
    forced_photometry_["e_brightness"] = mag_err_2_flux_err(
        forced_photometry_["e_mag"], forced_photometry_["mag"]
    )

    forced_photometry_["candid"] = forced_photometry_["oid"].astype(str) + forced_photometry_["pid"].astype(str) 

    return forced_photometry_

def preprocess_detections_and_forced_photometry_from_db(
    detections: pd.DataFrame, forced_photometry: pd.DataFrame
) -> pd.DataFrame:
    columns = ['oid', 'mjd']
    print(f'detections:\n{detections[columns]}') 
    print(f'forced_photometry:\n{forced_photometry[columns]}') 

    detections_and_forced_photometry = pd.concat(
        [
            preprocess_detections_from_db(detections),
            preprocess_forced_photometry_from_db(forced_photometry),
        ]
    )

    detections_and_forced_photometry = get_det_AO_to_InputDTO(detections_and_forced_photometry)
    print(f'1. detections_and_forced_photometry:\n{detections_and_forced_photometry[columns]}') 
    detections_and_forced_photometry.drop_duplicates(["candid", "oid"], inplace=True)
    print(f'2. detections_and_forced_photometry:\n{detections_and_forced_photometry[columns]}')
    detections_and_forced_photometry.sort_values(
        by=["mjd", "forced"], ascending=[True, False], inplace=True
    )
    print(f'3. detections_and_forced_photometry:\n{detections_and_forced_photometry[columns]}')
    detections_and_forced_photometry.set_index("oid", inplace=True)
    return detections_and_forced_photometry

In [211]:
def preprocess_features_from_db(features: pd.DataFrame) -> pd.DataFrame:
    """features come from db in column format"""
    def fid_map(v):
        if v == 1:
            return "_1"  #'g'
        if v == 2:
            return "_2"  #'r'
        if v == 0:
            return ""
        
        if v == 12:
            return "_12"

    def normalize_name(v):
        return str(v).replace("-", "_").replace('/', '_')

    features_ = features.copy()
    features_["fid_name"] = features_["fid"].apply(lambda x: fid_map(x))
    features_["name"] = features_["name"] + features_["fid_name"]
    features_["name"] = features_["name"].apply(lambda x: normalize_name(x))
    features_ = features_[["oid", "name", "value"]]
    features_ = features_.pivot_table(
        index="oid", columns="name", values="value", aggfunc="first", dropna=False
    )

    return features_

In [212]:
from alerce_classifiers.base.dto import InputDTO
from alerce_classifiers.base._types import *

def transform_dataframes_to_input_dto(data: dict) -> InputDTO:
    """get dataframes from detections and forced photometry"""

    detections_and_forced_photometry = (
        preprocess_detections_and_forced_photometry_from_db(
            data["detections"], data["forced_photometry"]
        )
    )

    features = preprocess_features_from_db(data["features"])
    """ format dataframe of features """

    return InputDTO(
        Detections(detections_and_forced_photometry),
        NonDetections(pd.DataFrame()),
        Features(pd.DataFrame(features)),
        Xmatch(pd.DataFrame()),
        Stamps(pd.DataFrame()),
    )

In [214]:
inspector = inspect(engine)
columns = inspector.get_columns("object")

for column in columns:
    print(column["name"], column["type"])

oid VARCHAR
ndethist INTEGER
ncovhist INTEGER
mjdstarthist DOUBLE PRECISION
mjdendhist DOUBLE PRECISION
corrected BOOLEAN
stellar BOOLEAN
ndet INTEGER
g_r_max DOUBLE PRECISION
g_r_max_corr DOUBLE PRECISION
g_r_mean DOUBLE PRECISION
g_r_mean_corr DOUBLE PRECISION
meanra DOUBLE PRECISION
meandec DOUBLE PRECISION
sigmara DOUBLE PRECISION
sigmadec DOUBLE PRECISION
deltajd DOUBLE PRECISION
firstmjd DOUBLE PRECISION
lastmjd DOUBLE PRECISION
step_id_corr VARCHAR
diffpos BOOLEAN
reference_change BOOLEAN


In [215]:
input_dto = transform_dataframes_to_input_dto(data_dict)
input_dto

detections:
            oid           mjd
0  ZTF18acbvugo  60727.215776
1  ZTF18acbvugo  60725.194618
2  ZTF18acbvugo  60725.139537
3  ZTF18acbvugo  60723.161215
4  ZTF18acbvugo  60751.260232
5  ZTF18acbvugo  60751.196481
6  ZTF18acbvugo  60714.386157
7  ZTF18acbvugo  60706.274896
8  ZTF18acbvugo  60733.286678
forced_photometry:
            oid           mjd
0  ZTF18acbvugo  60744.276586
1  ZTF18acbvugo  60739.130266
2  ZTF18acbvugo  60733.286678
3  ZTF18acbvugo  60731.200243
4  ZTF18acbvugo  60731.139190
5  ZTF18acbvugo  60729.197732
6  ZTF18acbvugo  60729.166609
7  ZTF18acbvugo  60751.196481
1. detections_and_forced_photometry:
            oid           mjd
0  ZTF18acbvugo  60727.215776
1  ZTF18acbvugo  60725.194618
2  ZTF18acbvugo  60725.139537
3  ZTF18acbvugo  60723.161215
4  ZTF18acbvugo  60751.260232
5  ZTF18acbvugo  60751.196481
6  ZTF18acbvugo  60714.386157
7  ZTF18acbvugo  60706.274896
8  ZTF18acbvugo  60733.286678
0  ZTF18acbvugo  60744.276586
1  ZTF18acbvugo  60739.130266
2 

/tmp/ipykernel_2365004/795736498.py:104: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  forced_photometry_.fillna(value=np.nan, inplace=True)


InputDTO(_detections=Detections(_value=                                 candid           mjd  fid           pid  \
oid                                                                        
ZTF18acbvugo        2952274895215010005  60706.274896    2  2.952275e+12   
ZTF18acbvugo        2960386155215015101  60714.386157    1  2.960386e+12   
ZTF18acbvugo        2969161215215010001  60723.161215    1  2.969161e+12   
ZTF18acbvugo        2971139535215015005  60725.139537    2  2.971140e+12   
ZTF18acbvugo        2971194615215015015  60725.194618    1  2.971195e+12   
ZTF18acbvugo        2973215775215010006  60727.215776    2  2.973216e+12   
ZTF18acbvugo  ZTF18acbvugo2975166605215  60729.166609    2  2.975167e+12   
ZTF18acbvugo  ZTF18acbvugo2975197735215  60729.197732    1  2.975198e+12   
ZTF18acbvugo  ZTF18acbvugo2977139185215  60731.139190    1  2.977139e+12   
ZTF18acbvugo  ZTF18acbvugo2977200245215  60731.200243    2  2.977200e+12   
ZTF18acbvugo        2979286675215010001  60733.28

In [216]:
input_dto.features

name,Amplitude_1,Amplitude_2,AndersonDarling_1,AndersonDarling_2,Autocor_length_1,Autocor_length_2,Beyond1Std_1,Beyond1Std_2,Con_1,Con_2,...,ulens_fs_1,ulens_fs_2,ulens_mag0_1,ulens_mag0_2,ulens_t0_1,ulens_t0_2,ulens_tE_1,ulens_tE_2,ulens_u0_1,ulens_u0_2
oid,,,,,,,,,,,,,,,,,,,,,
ZTF18acbvugo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.501693,1.0,16.973457,16.166757,-77.597622,23.966545,11.214725,0.547382,152.698871,6.907979e-08


In [217]:
from alerce_classifiers.mbappe.mapper import MbappeMapper
from alerce_classifiers.mbappe.model import MbappeClassifier

MBAPPE_MODEL_PATH = "https://alerce-models.s3.amazonaws.com/mbappe/0.3.5/model.ckpt"
MBAPPE_QUANTILES_PATH = "https://alerce-models.s3.amazonaws.com/mbappe/0.3.5/quantiles"
MBAPPE_CONFIG_PATH = "https://alerce-models.s3.amazonaws.com/mbappe/0.3.5/configs"

model = MbappeClassifier(
    model_path=MBAPPE_MODEL_PATH,
    quantiles_dir=MBAPPE_QUANTILES_PATH,
    config_dir=MBAPPE_CONFIG_PATH,
    mapper=MbappeMapper(),
)

predictions = model.predict(input_dto)
pred_prob_mbappe = predictions.probabilities
pred_prob_mbappe = pred_prob_mbappe.reset_index()
pred_prob_mbappe

/home/dmoreno/pipeline_v4_final/pipeline/alerce_classifiers/alerce_classifiers/mbappe/mapper.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.loc[:, self.dict_info["md_cols"]] = quantiles["quantile_md"].transform(
/home/dmoreno/pipeline_v4_final/pipeline/alerce_classifiers/alerce_classifiers/mbappe/mapper.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.loc[:, self.dict_info["feat_cols"]] = quantiles[


,oid,CEP,AGN,LPV,SNIa,EB/EW,CV/Nova,EA,QSO,YSO,...,RSCVn,RRLab,DSCT,Periodic-Other,Microlensing,SLSN,SNIIn,SNIbc,SNIIb,TDE
0,ZTF18acbvugo,0.006516,0.000005,0.000077,7.798994e-08,0.366193,0.003991,0.51907,3.736879e-07,0.061025,...,0.000015,0.036521,0.000019,5.185304e-07,0.000005,8.133073e-09,7.872848e-10,2.971036e-08,5.998330e-11,1.352231e-07


In [218]:
df_long = pred_prob_mbappe.melt(id_vars=['oid'], var_name='class_name', value_name='probability')

# Asignar ranking por objeto (mayor probabilidad es 1)
df_long['ranking'] = df_long.groupby('oid')['probability'].rank(method="first", ascending=False).astype(int)
df_long


,oid,class_name,probability,ranking
0,ZTF18acbvugo,CEP,6.515733e-03,6
1,ZTF18acbvugo,AGN,5.488875e-06,12
2,ZTF18acbvugo,LPV,7.720343e-05,8
3,ZTF18acbvugo,SNIa,7.798994e-08,17
4,ZTF18acbvugo,EB/EW,3.661930e-01,2
5,ZTF18acbvugo,CV/Nova,3.991480e-03,7
6,ZTF18acbvugo,EA,5.190703e-01,1
7,ZTF18acbvugo,QSO,3.736879e-07,15
8,ZTF18acbvugo,YSO,6.102506e-02,3
9,ZTF18acbvugo,SNII,3.296341e-09,20


In [219]:
query_to_prod_atat = f"""
            SELECT * FROM probability as p
            WHERE oid  in ({','.join(oids)})
                    and p.classifier_name = 'LC_classifier_ATAT_forced_phot' 
                    and classifier_version = '0.3.5';
        """
prod_probs_df_atat = pd.read_sql_query(query_to_prod_atat, con=engine)[df_long.columns]
prod_probs_df_atat

,oid,class_name,probability,ranking
0,ZTF18acbvugo,AGN,0.001047,14
1,ZTF18acbvugo,Blazar,0.003600,11
2,ZTF18acbvugo,CEP,0.024473,7
3,ZTF18acbvugo,CV/Nova,0.045123,6
4,ZTF18acbvugo,DSCT,0.007715,10
5,ZTF18acbvugo,EA,0.125950,3
6,ZTF18acbvugo,EB/EW,0.222332,2
7,ZTF18acbvugo,LPV,0.002224,12
8,ZTF18acbvugo,Microlensing,0.000056,16
9,ZTF18acbvugo,Periodic-Other,0.009848,9


In [220]:
df_comparacion = df_long.merge(prod_probs_df_atat, on=['oid', 'class_name'], suffixes=('_local', '_prod'))
df_comparacion

,oid,class_name,probability_local,ranking_local,probability_prod,ranking_prod
0,ZTF18acbvugo,CEP,6.515733e-03,6,0.024473,7
1,ZTF18acbvugo,AGN,5.488875e-06,12,0.001047,14
2,ZTF18acbvugo,LPV,7.720343e-05,8,0.002224,12
3,ZTF18acbvugo,SNIa,7.798994e-08,17,0.000008,20
4,ZTF18acbvugo,EB/EW,3.661930e-01,2,0.222332,2
5,ZTF18acbvugo,CV/Nova,3.991480e-03,7,0.045123,6
6,ZTF18acbvugo,EA,5.190703e-01,1,0.125950,3
7,ZTF18acbvugo,QSO,3.736879e-07,15,0.001427,13
8,ZTF18acbvugo,YSO,6.102506e-02,3,0.392189,1
9,ZTF18acbvugo,SNII,3.296341e-09,20,0.000015,18


In [221]:
df_comparacion['ranking_diff'] = df_comparacion['ranking_local'] - df_comparacion['ranking_prod']
df_comparacion['probability_diff'] = df_comparacion['probability_local'] - df_comparacion['probability_prod']
df_comparacion

,oid,class_name,probability_local,ranking_local,probability_prod,ranking_prod,ranking_diff,probability_diff
0,ZTF18acbvugo,CEP,6.515733e-03,6,0.024473,7,-1,-0.017957
1,ZTF18acbvugo,AGN,5.488875e-06,12,0.001047,14,-2,-0.001042
2,ZTF18acbvugo,LPV,7.720343e-05,8,0.002224,12,-4,-0.002147
3,ZTF18acbvugo,SNIa,7.798994e-08,17,0.000008,20,-3,-0.000008
4,ZTF18acbvugo,EB/EW,3.661930e-01,2,0.222332,2,0,0.143861
5,ZTF18acbvugo,CV/Nova,3.991480e-03,7,0.045123,6,1,-0.041132
6,ZTF18acbvugo,EA,5.190703e-01,1,0.125950,3,-2,0.393120
7,ZTF18acbvugo,QSO,3.736879e-07,15,0.001427,13,2,-0.001426
8,ZTF18acbvugo,YSO,6.102506e-02,3,0.392189,1,2,-0.331164
9,ZTF18acbvugo,SNII,3.296341e-09,20,0.000015,18,2,-0.000015


In [222]:
df_comparacion[df_comparacion['ranking_local'] <= 3]

,oid,class_name,probability_local,ranking_local,probability_prod,ranking_prod,ranking_diff,probability_diff
4,ZTF18acbvugo,EB/EW,0.366193,2,0.222332,2,0,0.143861
6,ZTF18acbvugo,EA,0.519070,1,0.125950,3,-2,0.393120
8,ZTF18acbvugo,YSO,0.061025,3,0.392189,1,2,-0.331164


In [74]:
prod_probs_df_atat.classifier_version.unique()

array(['1.1.4', 'lc_classifier_1.1.13', '0.3.5', '0.0.0'], dtype=object)

In [77]:
prod_probs_df_atat.classifier_name.unique()

array(['LC_classifier_BHRF_forced_phot', 'lc_classifier',
       'LC_classifier_BHRF_forced_phot_periodic',
       'LC_classifier_BHRF_forced_phot_stochastic',
       'LC_classifier_ATAT_forced_phot',
       'LC_classifier_BHRF_forced_phot_transient',
       'LC_classifier_BHRF_forced_phot_top', 'STAMP_2025_beta'],
      dtype=object)

In [ ]:


ao = create_astro_object(
    data_origin='database',
    detections=detections,
    forced_photometry=forced_photometry,
    xmatch=None,
    reference=None,
    non_detections=None,
)

In [14]:
#oids = ["ZTF18aabgipf", "ZTF18aavtats"]
oids = ['ZTF20aclvqmx']
oids = [f"'{oid}'" for oid in oids]
#oids_str = ", ".join(f"'{oid}'" for oid in oids) 

In [15]:
query_detections = f"""
    SELECT * FROM detection
    WHERE oid IN ({','.join(oids)}) and rb >= 0.55;
"""

detections = pd.read_sql_query(query_detections, con=engine)
detections

,oid,candid,mjd,fid,pid,diffmaglim,isdiffpos,nid,ra,dec,...,sigmagapbig,rfid,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext,corrected,dubious,parent_candid,has_stamp,step_id_corr
0,ZTF20aclvqmx,2716363714315015011,60470.363715,2,2716363714315,20.411406,1,2716,312.645362,11.953960,...,0.2394,5.431202e+08,18.124825,0.027930,0.031430,True,False,NaN,True,24.5.1
1,ZTF20aclvqmx,2718439834315015017,60472.439838,1,2718439834315,20.633200,1,2718,312.645267,11.953988,...,0.3947,NaN,19.084934,0.056601,0.058385,True,False,2.736391e+18,False,24.5.1
2,ZTF20aclvqmx,2722456904315015018,60476.456910,2,2722456904315,20.593800,1,2722,312.645356,11.954091,...,0.2158,NaN,18.108067,0.033905,0.036756,True,False,2.736391e+18,False,24.5.1
3,ZTF20aclvqmx,2725359274315015009,60479.359271,2,2725359274315,20.023352,1,2725,312.645291,11.954002,...,0.4066,5.431202e+08,18.116410,0.030457,0.033648,True,False,NaN,True,24.5.1
4,ZTF20aclvqmx,2736391454315015023,60490.391458,2,2736391454315,20.462019,1,2736,312.645366,11.954054,...,0.2747,5.431202e+08,18.122684,0.023093,0.027207,True,False,NaN,True,24.5.1
5,ZTF20aclvqmx,2740358994315015015,60494.358993,2,2740358994315,20.536700,1,2740,312.645397,11.954088,...,0.3566,NaN,18.141165,0.034247,0.037242,True,False,2.753358e+18,False,24.5.1
6,ZTF20aclvqmx,2753358264315015016,60507.358264,2,2753358264315,20.466166,1,2753,312.645331,11.954049,...,0.2636,5.431202e+08,18.113300,0.022522,0.026658,True,False,NaN,True,24.5.1
7,ZTF20aclvqmx,2784264004315025034,60538.264005,2,2784264004315,19.814900,1,2784,312.645415,11.954071,...,0.4117,NaN,18.080547,0.043095,0.045262,True,False,2.805232e+18,False,24.5.1
8,ZTF20aclvqmx,2795275934315015012,60549.275937,1,2795275934315,20.572600,1,2795,312.645396,11.954119,...,0.3597,NaN,19.037806,0.061131,0.062651,True,False,2.824193e+18,False,24.5.1
9,ZTF20aclvqmx,2803261404315015011,60557.261400,1,2803261404315,20.607700,1,2803,312.645469,11.954107,...,0.5656,NaN,19.028149,0.063716,0.065150,True,False,2.824193e+18,False,24.5.1


In [11]:
for col in detections.columns:
    print(col)

candid
oid
mjd
fid
pid
diffmaglim
isdiffpos
nid
ra
dec
magpsf
sigmapsf
magap
sigmagap
distnr
rb
rbversion
drb
drbversion
magapbig
sigmagapbig
rfid
magpsf_corr
sigmapsf_corr
sigmapsf_corr_ext
corrected
dubious
parent_candid
has_stamp
step_id_corr


In [12]:
# FORCED PHOTOMETRY
query_forced_photometry = f"""
                    SELECT * FROM forced_photometry
                    WHERE oid in ({','.join(oids)}) and procstatus in ('0', '57');
                    """

# and procstatus in ('0', '57') ;
# Por que aplica el filtro procstatus entre 0 y 57?
forced_photometry = pd.read_sql_query(query_forced_photometry, con=engine)
forced_photometry

,pid,oid,mjd,fid,ra,dec,e_ra,e_dec,mag,e_mag,...,diffmaglim,programid,procstatus,distnr,ranr,decnr,magnr,sigmagnr,chinr,sharpnr
0,2973560900615,ZTF20aclvqmx,60727.560903,2,312.645314,11.954185,None,None,19.77495,0.519044,...,18.2817,1,0,0.304448,312.645294,11.954106,18.334,0.019,0.477,0.02


In [48]:
for col in forced_photometry.columns:
    print(col)

pid
oid
mjd
fid
ra
dec
e_ra
e_dec
mag
e_mag
mag_corr
e_mag_corr
e_mag_corr_ext
isdiffpos
corrected
dubious
parent_candid
has_stamp
field
rcid
rfid
sciinpseeing
scibckgnd
scisigpix
magzpsci
magzpsciunc
magzpscirms
clrcoeff
clrcounc
exptime
adpctdif1
adpctdif2
diffmaglim
programid
procstatus
distnr
ranr
decnr
magnr
sigmagnr
chinr
sharpnr


In [49]:
lc_det_ff = pd.concat([detections, forced_photometry])
lc_det_ff

,oid,candid,mjd,fid,pid,diffmaglim,isdiffpos,nid,ra,dec,...,adpctdif1,adpctdif2,programid,procstatus,ranr,decnr,magnr,sigmagnr,chinr,sharpnr
0,ZTF20aclvqmx,2.716364e+18,60470.363715,2,2716363714315,20.411406,1,2716.0,312.645362,11.953960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZTF20aclvqmx,2.718440e+18,60472.439838,1,2718439834315,20.633200,1,2718.0,312.645267,11.953988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZTF20aclvqmx,2.722457e+18,60476.456910,2,2722456904315,20.593800,1,2722.0,312.645356,11.954091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZTF20aclvqmx,2.725359e+18,60479.359271,2,2725359274315,20.023352,1,2725.0,312.645291,11.954002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZTF20aclvqmx,2.736391e+18,60490.391458,2,2736391454315,20.462019,1,2736.0,312.645366,11.954054,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,ZTF20aclvqmx,NaN,60534.252222,1,2780252224315,20.719299,1,NaN,312.645416,11.954047,...,0.053127,0.052644,1.0,0,312.645264,11.954123,19.333000,0.018,0.587,-0.009
104,ZTF20aclvqmx,NaN,60532.323287,2,2778323284315,20.914801,1,NaN,312.645416,11.954047,...,0.052043,0.058391,1.0,0,312.645264,11.954128,18.303999,0.017,0.504,0.023
105,ZTF20aclvqmx,NaN,60532.253276,1,2778253274315,20.866301,1,NaN,312.645416,11.954047,...,0.054988,0.054125,1.0,0,312.645264,11.954123,19.333000,0.018,0.587,-0.009
106,ZTF20aclvqmx,NaN,60531.331169,1,2777331164315,20.761600,1,NaN,312.645416,11.954047,...,0.059938,0.061376,1.0,0,312.645264,11.954123,19.333000,0.018,0.587,-0.009


In [50]:
lc_det_ff_with_drop = lc_det_ff.drop_duplicates(
    ["candid", "oid"], keep="first"
)
lc_det_ff_with_drop

,oid,candid,mjd,fid,pid,diffmaglim,isdiffpos,nid,ra,dec,...,adpctdif1,adpctdif2,programid,procstatus,ranr,decnr,magnr,sigmagnr,chinr,sharpnr
0,ZTF20aclvqmx,2.716364e+18,60470.363715,2,2716363714315,20.411406,1,2716.0,312.645362,11.953960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZTF20aclvqmx,2.718440e+18,60472.439838,1,2718439834315,20.633200,1,2718.0,312.645267,11.953988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ZTF20aclvqmx,2.722457e+18,60476.456910,2,2722456904315,20.593800,1,2722.0,312.645356,11.954091,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ZTF20aclvqmx,2.725359e+18,60479.359271,2,2725359274315,20.023352,1,2725.0,312.645291,11.954002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ZTF20aclvqmx,2.736391e+18,60490.391458,2,2736391454315,20.462019,1,2736.0,312.645366,11.954054,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ZTF20aclvqmx,2.740359e+18,60494.358993,2,2740358994315,20.536700,1,2740.0,312.645397,11.954088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ZTF20aclvqmx,2.753358e+18,60507.358264,2,2753358264315,20.466166,1,2753.0,312.645331,11.954049,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ZTF20aclvqmx,2.784264e+18,60538.264005,2,2784264004315,19.814900,1,2784.0,312.645415,11.954071,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ZTF20aclvqmx,2.795276e+18,60549.275937,1,2795275934315,20.572600,1,2795.0,312.645396,11.954119,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ZTF20aclvqmx,2.803261e+18,60557.261400,1,2803261404315,20.607700,1,2803.0,312.645469,11.954107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Compute Features

In [54]:
def patch_wise(oid_list):
    columns = ["oid", "w1mpro", "w2mpro", "w3mpro", "w4mpro"]
    df = pd.DataFrame({"oid": oid_list})  # Fill only the oid column
    for col in columns[1:]:  # Add empty columns for the rest
        df[col] = None
    return df

def features_from_db(features: pd.DataFrame):
    def normalize_name(x: str | int):
        if "_0" in x:
            return x[:-2].replace("/", "_")
        return x.replace("/", "_")

    def normalize_fid(x):
        if x == 0:
            return None
        if x == 1:
            return "g"
        if x == 2:
            return "r"
        if x == 12:
            return "g,r"

    # features['name'] = features['name'] + '_' + features['fid'].astype(str)
    if len(features) >= 1:
        features["name"] = features.apply(lambda x: normalize_name(x["name"]), axis=1)
        features["fid"] = features.apply(lambda x: normalize_fid(x["fid"]), axis=1)
        return features
    else:
        return None

In [1]:
# GET FEATURES
query_features = f"""
                    SELECT * FROM feature as f 
                    WHERE f.oid in ({','.join(oids)}) and f.version = '27.3.0';
                    """

features = pd.read_sql_query(query_features, con=engine)
features

NameError: name 'oids' is not defined

In [56]:
features = features_from_db(features)
features

,oid,name,value,fid,version
0,ZTF20aclvqmx,Amplitude,NaN,g,27.4.0
1,ZTF20aclvqmx,Amplitude,0.055532,r,27.4.0
2,ZTF20aclvqmx,AndersonDarling,NaN,g,27.4.0
3,ZTF20aclvqmx,AndersonDarling,0.999087,r,27.4.0
4,ZTF20aclvqmx,Autocor_length,NaN,g,27.4.0
...,...,...,...,...,...
204,ZTF20aclvqmx,ulens_u0,0.140078,g,27.4.0
205,ZTF20aclvqmx,ulens_u0,0.000853,r,27.4.0
206,ZTF20aclvqmx,W1-W2,NaN,None,27.4.0
207,ZTF20aclvqmx,W2-W3,NaN,None,27.4.0


In [58]:
from lc_classifier.utils import (
    #all_features_from_astro_objects,
    create_astro_object,
    #EmptyLightcurveException,
)

ImportError: Numba needs NumPy 1.26 or less

In [ ]:
ao = create_astro_object(
    data_origin='database',
    detections=detections,
    forced_photometry=forced_photometry,
    xmatch=None,
    reference=None,
    non_detections=None,
)

ao

In [ ]:
from lc_classifier.features.preprocess.ztf import (
    ZTFLightcurvePreprocessor,
)
from lc_classifier.features.composites.ztf import ZTFFeatureExtractor

lightcurve_preprocessor = ZTFLightcurvePreprocessor(drop_bogus=False)
lightcurve_preprocessor.preprocess_batch(batch_astro_objects)

In [ ]:
def dataframes_to_astro_object_list(
    detections,
    forced_photometry,
    xmatch,
    reference,
    features=None,
    data_origin="database",
    verbose=True,
):
    oids = detections["oid"].unique()
    detections.set_index("oid", inplace=True)
    detections.sort_index(inplace=True)
    forced_photometry_columns = forced_photometry.columns
    forced_photometry.set_index("oid", inplace=True)
    forced_photometry.sort_index(inplace=True)
    xmatch.set_index("oid", inplace=True)
    reference_columns = reference.columns
    reference.set_index("oid", inplace=True)
    astro_objects_list = []
    for oid in oids:
        try:
            xmatch_oid = xmatch.loc[[oid]].reset_index()
        except:
            logging.warning("xmatch metadata patched!")
            xmatch_oid = patch_xmatch_by_oid(oid=oid).reset_index()

        assert len(xmatch_oid) == 1
        xmatch_oid = xmatch_oid.iloc[0]
        if oid in forced_photometry.index:
            forced_photometry_oid = forced_photometry.loc[[oid]].reset_index()
        else:
            forced_photometry_oid = pd.DataFrame(columns=forced_photometry_columns)
        if oid in reference.index:
            reference_oid = reference.loc[[oid]].reset_index()
        else:
            reference_oid = pd.DataFrame(columns=reference_columns)
        try:
            ao = create_astro_object(
                data_origin=data_origin,
                detections=detections.loc[[oid]].reset_index(),
                forced_photometry=forced_photometry_oid,
                xmatch=xmatch_oid,
                reference=reference_oid,
                non_detections=None,
            )
            if features is not None:
                """add features from db"""
                try:
                    ao.features = features.loc[features.oid == oid][
                        ["name", "value", "fid", "version"]
                    ]
                except:
                    ao.features = None
        except EmptyLightcurveException:
            continue

        astro_objects_list.append(ao)
    return astro_objects_list

In [ ]:
engine = sa.create_engine(
    f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}"
)
engine.begin()


""" QUERIES """
# DETECTION
query_detections = f"""
                    SELECT * FROM detection
                    WHERE oid in ({oids_str}) and rb >= 0.55;
                    """
detections = pd.read_sql_query(query_detections, con=engine)

# FORCED PHOTOMETRY
query_forced_photometry = f"""
                    SELECT * FROM forced_photometry
                    WHERE oid in ({oids_str}) and procstatus in ('0', '57');
                    """
forced_photometry = pd.read_sql_query(query_forced_photometry, con=engine)

# REFERENCE
query_reference = f"""
                    SELECT oid, rfid, sharpnr, chinr FROM reference
                    WHERE oid in ({oids_str}) and chinr >= 0;
                    """
reference = pd.read_sql_query(query_reference, con=engine)
reference = reference.drop_duplicates("rfid")

# XMATCH
query_xmatch = f"""
                SELECT oid, oid_catalog, dist FROM xmatch
                WHERE oid in ({oids_str}) and catid='allwise';
                """
xmatch = pd.read_sql_query(query_xmatch, con=engine)
xmatch = xmatch.sort_values("dist").drop_duplicates("oid")
oid_catalog = [f"'{oid}'" for oid in xmatch["oid_catalog"].values]

# ALL WISE
query_wise = f"""
            SELECT oid_catalog, w1mpro, w2mpro, w3mpro, w4mpro FROM allwise
            WHERE oid_catalog in ({','.join(oid_catalog)});
            """
try:
    wise = pd.read_sql_query(query_wise, con=engine).set_index("oid_catalog")
    wise = pd.merge(xmatch, wise, on="oid_catalog", how="outer")
    wise = wise[["oid", "w1mpro", "w2mpro", "w3mpro", "w4mpro"]].set_index("oid")
except:
    wise = patch_wise(oids)
    logging.warning("wise patched")

# PANSTARR
query_ps = f"""
            SELECT oid, sgscore1, sgmag1, srmag1, simag1, szmag1, distpsnr1 FROM ps1_ztf
            WHERE oid in ({oids_str});
            """
ps = pd.read_sql_query(query_ps, con=engine)
ps = ps.drop_duplicates("oid").set_index("oid")

#
xmatch = pd.concat([wise, ps], axis=1).reset_index()

# GET FEATURES
query_features = f"""
                    SELECT  * FROM feature as f 
                    WHERE f.oid  in ({oids_str}) and f.version = '27.3.0';
                    """
features = pd.read_sql_query(query_features, con=engine)
features = features_from_db(features)

# CREATE ASTRO OBJECTS FROM DATAFRAMES EXTRATED FROM DATABASE
output = dataframes_to_astro_object_list(
    detections=detections,
    forced_photometry=forced_photometry,
    xmatch=xmatch,
    reference=reference,
    features=features,
)

In [ ]:
from lc_classifier.features.preprocess.ztf import ZTFLightcurvePreprocessor
from lc_classifier.features.composites.ztf import ZTFFeatureExtractor
from lc_classifier.features.core.base import astro_object_from_dict

In [ ]:
lightcurve_preprocessor = ZTFLightcurvePreprocessor(drop_bogus=False)
lightcurve_preprocessor.preprocess_batch(batch_astro_objects)
feature_extractor = ZTFFeatureExtractor()
feature_extractor.compute_features_batch(batch_astro_objects, progress_bar=False)

In [17]:
# GET FEATURES
query_features = f"""
                    SELECT  * FROM feature as f 
                    WHERE f.oid  in ({oids_str}) and f.version = '27.4.0';
                    """

features = pd.read_sql_query(query_features, con=engine)
features

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "ZTF18aabgipf"
LINE 3:                     WHERE f.oid  in (''ZTF18aabgipf'', ''ZTF...
                                               ^

[SQL: 
                    SELECT  * FROM feature as f 
                    WHERE f.oid  in (''ZTF18aabgipf'', ''ZTF18aavtats'') and f.version = '27.4.0';
                    ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [9]:
RANKING = 1

query_to_prod_bhrf = f"""
                SELECT  * FROM alerce.probability as p
                WHERE oid  in ({oids_str})
                        and p.classifier_name = 'LC_classifier_BHRF_forced_phot(beta)' 
                        and p.ranking = {RANKING} 
                        and classifier_version = '1.1.5';
            """

query_to_prod_atat = f"""
                SELECT  * FROM alerce.probability as p
                WHERE oid  in ({oids_str})
                        and p.classifier_name = 'LC_classifier_ATAT_forced_phot(beta)' 
                        and p.ranking = {RANKING} 
                        and classifier_version = '0.3.6';
            """

engine = sa.create_engine('postgresql+psycopg2://' + params['user'] \
                          + ':' + params['password'] + '@' + params['host'] \
                          + '/' + params['dbname'])
engine.begin()

prod_probs_df_bhrf = pd.read_sql_query(query_to_prod_bhrf, con=engine)
prod_probs_df_atat = pd.read_sql_query(query_to_prod_atat, con=engine)

In [10]:
prod_probs_df_atat

,oid,classifier_name,classifier_version,class_name,probability,ranking
0,ZTF18aabgipf,LC_classifier_ATAT_forced_phot(beta),0.3.6,LPV,0.893018,1


In [11]:
prod_probs_df_bhrf

,oid,classifier_name,classifier_version,class_name,probability,ranking
0,ZTF18aabgipf,LC_classifier_BHRF_forced_phot(beta),1.1.5,LPV,0.689308,1


In [ ]:
a = ['ZTF17aaaafww',
 'ZTF17aaaagbs',
 'ZTF17aaaagvz',
 'ZTF17aaaagwz',
 'ZTF17aaaaotk',
 'ZTF17aaabate',
 'ZTF17aaabdxt',
 'ZTF17aaabebp',
 'ZTF17aaabebt',
 'ZTF17aaabwrg',
 'ZTF17aaacnht',
 'ZTF17aaaewdz',
 'ZTF17aaagezb',
 'ZTF17aaagtow',
 'ZTF17aaagypl',
 'ZTF17aaahegx',
 'ZTF17aaahejg',
 'ZTF17aaahere',
 'ZTF17aaahixx',
 'ZTF17aaahtvd',
 'ZTF17aaaiedz',
 'ZTF17aaaiegd',
 'ZTF17aaaignl',
 'ZTF17aaaihls',
 'ZTF17aaaihlz']


# Esos no dan iguales
b = ['ZTF17aaaewpk',
 'ZTF17aabrocy',
 'ZTF18aaaatqy',
 'ZTF18aaabter',
 'ZTF18aaakkxx',
 'ZTF18ablmuyz',
 'ZTF18absceqp',
 'ZTF18abzrmcq']
 

In [5]:
environment_variables = {
    "POSTGRES_USER": "postgres",
    "POSTGRES_PASSWORD": "postgres",
    "POSTGRES_DB": "ztf",
    "POSTGRES_PORT": 5432,
    "POSTGRES_HOST": "localhost",
    "POSTGRES_SCHEMA": "alerce",
    "N_JOBS": 16,
    "CONTAINER_NAME": "psql_docker",
    "CONTAINER_VOLUME_NAME": "integration_psql_data",
    "ASTRO_OBJECTS_PATH": "./batch/data",
    "ASTRO_OBJECTS_PATH_OUTPUT": "./batch/output",
}

In [15]:
import psycopg2

In [20]:
environment_variables = {
    "USER":"alerce",
    "PASSWORD":"IG,eFKHi5[",
    "HOST":"30.0.85.190",
    "PORT":"5432",
    "DB_NAME":"ztf"
    }

In [18]:
conn = psycopg2.connect(
    host=environment_variables['HOST'],
    port=environment_variables['PORT'],
    user=environment_variables['USER'],
    password=environment_variables['PASSWORD'],
    database=environment_variables['DB_NAME']
)

#lightcurves = pd.read_sql(query, conn)

OperationalError: connection to server at "30.0.85.190", port 5432 failed: Connection timed out
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
# Lista de OIDs y ranking
oids = ['ZTF17aaaiedz'] #, 'oid2', 'oid3']  # Reemplaza con tus OIDs
RANKING = 1  # Ajusta según sea necesario

# Consulta segura con parámetros
query = """
    SELECT * FROM alerce.probability
    WHERE oid IN %(oids)s
    AND classifier_name = 'LC_classifier_ATAT_forced_phot(beta)' 
    AND ranking = %(ranking)s
    AND classifier_version = '0.3.4';
"""

# Ejecutar la consulta con parámetros
local_probs_df_atat = pd.read_sql(query, conn, params={'oids': tuple(oids), 'ranking': RANKING})

# Cerrar la conexión
conn.close()

In [ ]:
local_engine = sa.create_engine(
    f"postgresql+psycopg2://{environment_variables['POSTGRES_USER']}:{environment_variables['POSTGRES_PASSWORD']}@{environment_variables['POSTGRES_HOST']}/{environment_variables['POSTGRES_DB']}"
)

# Definir valores de detección
ndet_max = 50
ndet_min = 8

In [13]:
## Consulta SQL con parámetros seguros
#query = """
#    SELECT * FROM alerce.probability
#    WHERE classifier_name = 'LC_classifier_ATAT_forced_phot(beta)' 
#    AND ranking <= 3 
#    AND ndet BETWEEN %(ndet_min)s AND %(ndet_max)s
#"""

## Ejecutar la consulta con parámetros
#local_probs_df_atat = pd.read_sql(query, conn, params={'ndet_min': ndet_min, 'ndet_max': ndet_max})


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
ndet_max = 50
ndet_min = 8

query_to_local_atat = f"""
                SELECT * from alerce.probability as p
                        WHERE p.classifier_name  = 'LC_classifier_ATAT_forced_phot(beta)' 
                            and p.ranking <=3 and ndet <= {ndet_max} and ndet >= {ndet_min} 
                """

#local_probs_df_bhrf = pd.read_sql_query(query_to_local_bhrf, con=local_engine)
local_probs_df_atat = pd.read_sql_query(query_to_local_atat, con=local_engine)